Now that we have our trained models we can make some predictions and evaluate their peformance. There is no need to train our model's on the training data again since RandomizedSearchCV refits on the best estimator by default.

In [10]:
import joblib 
import pandas as pd
import numpy as np
import json
from sklearn.metrics import root_mean_squared_error
import sys
sys.path.append('..') #get root directory

In [12]:
svr_model = joblib.load('../models/svr_model.joblib')
rfr_model = joblib.load('../models/rfr_model.joblib')

with open('../models/svr_grid_model_score.json', 'r') as file:
    svr_log_score = json.load(file)
    
with open('../models/rfr_grid_model_score.json', 'r') as file:
    rfr_log_score = json.load(file)


In [13]:
X_test = pd.read_csv("../data/interim/X_test.csv", index_col= 0) # expects an untransformed X_train
y_test = pd.read_csv("../data/interim/y_test.csv", index_col = 0)
y_test_transformed = pd.read_csv("../data/processed/y_test_transformed.csv", index_col = 0)

Store the scoring values from the final models and compare to the log_prediction_rsme to see if model is underfitting or overfitting

In [ ]:
svr_predictions_log = svr_model.predict(X_test) # gives a prediction in log space
svr_predictions = np.exp(svr_predictions_log) 

svr_predictions_rsme = root_mean_squared_error(y_test, svr_predictions)
svr_log_predictions_rsme = root_mean_squared_error(y_test_transformed, svr_predictions_log)

print(f"SVR predictions RSME in dollars: ${svr_predictions_rsme}") # to get a 
print(f"SVR log predictions RSME: {svr_log_predictions_rsme}")
print(f"SVR Grid Search RSME: {svr_log_score}")

SVR predictions RSME in dollars: 201325.85312780892
SVR log predictions RSME: 0.2668818031828324
SVR Grid Search RSME: -0.29019480244638035


In [16]:
rfr_predictions_log = rfr_model.predict(X_test)
rfr_predictions = np.exp(rfr_predictions_log)

rfr_predictions_rsme = root_mean_squared_error(y_test, rfr_predictions)
rfr_predictions_log_rsme = root_mean_squared_error(y_test_transformed, rfr_predictions_log)

print(f"RFR predictions RSME in dollars: ${rfr_predictions_rsme}") # to get a 
print(f"RFR log predictions RSME: {rfr_predictions_log_rsme}")
print(f"RFR Grid Search RSME: {rfr_log_score}")

RFR predictions RSME in dollars: $240951.3127548868
RFR log predictions RSME: 0.27813922209746944
RFR Grid Search RSME: -0.30007231290261505
